#### Bootstrap OpenAssetIO

In [1]:
import json
import pprint

from openassetio import BatchElementException, TraitsData
from openassetio.hostApi import HostInterface, ManagerFactory
from openassetio.log import ConsoleLogger, SeverityFilter
from openassetio.pluginSystem import PythonPluginSystemManagerImplementationFactory

class DemoHostInterface(HostInterface):
    def identifier(self):
        return "org.demo.jupyter"

    def displayName(self):
        return "Demo Host"


logger = SeverityFilter(ConsoleLogger())
impl_factory = PythonPluginSystemManagerImplementationFactory(logger)
host_interface = DemoHostInterface()

manager = ManagerFactory.defaultManagerForInterface("resources/config.toml", host_interface, impl_factory, logger)
context = manager.createContext()
write_context = context # Fudge for BAL

def print_traits_data(data):
    as_dict = {
        trait_id: {
            property_key: data.getTraitProperty(trait_id, property_key)
            for property_key in data.traitPropertyKeys(trait_id)
        }
        for trait_id in data.traitSet()
    }
    print(json.dumps(as_dict, indent=4, sort_keys=True))


def take_picture(path):
    pass

def query_auth_token():
    return "RGF2aWQgaGFzIGdvZC1sZXZlbCBwZXJtaXNzaW9ucw=="

## Batch publish two different entity types

### Firstly, a user credential entity

(Note: bad example, but limited by traits available in MediaCreation)


In [2]:
from openassetio_mediacreation.traits.auth import BearerTokenTrait

credential_ref = manager.createEntityReference("bal:///demo_project/user/credential")

credential_data = TraitsData()
BearerTokenTrait.imbueTo(credential_data)

### Secondly, user's profile picture

We haven't yet taken the picture, but we know the user's name

In [3]:
from openassetio_mediacreation.traits.content import LocatableContentTrait
from openassetio_mediacreation.traits.identity import DisplayNameTrait

user_name = "David Feltell"
picture_ref = manager.createEntityReference("bal:///demo_project/user/profile/picture")

picture_data = TraitsData()
LocatableContentTrait.imbueTo(picture_data)
DisplayNameTrait.imbueTo(picture_data)

DisplayNameTrait(picture_data).setName(user_name)

### Preflight the entities

We now preflight the two entities, giving as much information up-front as possible.

Note that previously it was not possible to pass entities of different types (trait sets) along with properties known ahead-of-time.

In [4]:
# Note, previously: wk_refs = manager.preflight(refs, {"trait1", "trait2", ...}, context)

[picture_wk_ref, credential_wk_ref] = manager.preflight([picture_ref, credential_ref], [picture_data, credential_data], write_context)

### Populate the data for final register

User credentials first

In [5]:
# Get the authentication token
token = query_auth_token()
print(f"Token: {token}")

# Update the data to register with the manager. 
BearerTokenTrait(credential_data).setToken(token)

Token: RGF2aWQgaGFzIGdvZC1sZXZlbCBwZXJtaXNzaW9ucw==


Now user profile picture, making sure to ask the manager where it wants it to be saved.

Note that the file path is only possible thanks the the `preflight` API changes

In [6]:
# Get location the manager wants us to write to.
picture_location_data = manager.resolve(picture_wk_ref, {LocatableContentTrait.kId}, write_context)
picture_location = LocatableContentTrait(picture_location_data).getLocation()
print(f"Picture location: {picture_location}")

# Take a picture.
take_picture(picture_location)

# Update the data to register with the manager.
LocatableContentTrait(picture_data).setLocation(picture_location)

Picture location: file:///mnt/pictures/people/David%20Feltell.png


### Register the two (differently typed) entitites

In [7]:
[picture_final_ref, credential_final_ref] = manager.register([picture_wk_ref, credential_wk_ref], [picture_data, credential_data], context)

### Check they're registered OK

In [8]:
all_trait_ids = {LocatableContentTrait.kId, DisplayNameTrait.kId, BearerTokenTrait.kId}

[picture_final_data, credential_final_data] = manager.resolve([picture_final_ref, credential_final_ref], all_trait_ids, context)

print("Picture data")
print_traits_data(picture_final_data)
print("Credential data")
print_traits_data(credential_final_data)

Picture data
{
    "openassetio-mediacreation:content.LocatableContent": {
        "location": "file:///mnt/pictures/people/David%20Feltell.png"
    },
    "openassetio-mediacreation:identity.DisplayName": {
        "name": "David Feltell"
    }
}
Credential data
{
    "openassetio-mediacreation:auth.BearerToken": {
        "token": "RGF2aWQgaGFzIGdvZC1sZXZlbCBwZXJtaXNzaW9ucw=="
    }
}
